In questo Notebook ho introdotto la divisione in fasce orarie

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer
import ast


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


In [5]:
#il seguente vettore è stato ottenuto in analisi fatte precedentemente. 
SFancisco_stations = [39, 41, 42, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 
                      61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 82]

### Calcolo la distanza tra le diverse stazioni della città di San Francisco

In [6]:
#function to retrieve distance between 2 stations
def getDistance(lat_a, long_a, lat_b, long_b):
    # approximate radius of earth in km
    R = 6373.0    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [7]:
coordinates_stations_df = stations_df[stations_df["id"].isin(SFancisco_stations)]
coordinates_stations_df = coordinates_stations_df[["id", "lat", "long"]]
records = coordinates_stations_df.to_records(index=False)
list_coo = list(records)
# print(list_coo)

voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        lat_i=float(list_coo[i][1])
        long_i=float(list_coo[i][2])
        lat_j=float(list_coo[j][1])
        long_j=float(list_coo[j][2])
        distance=getDistance(lat_i, long_i, lat_j, long_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance


In [9]:
# voc_distances

In [8]:
distance_dictionary = {}


for station_id in SFancisco_stations:
    distance_dictionary[station_id] = []
    
    delta0_stations = []
    delta1_stations = []
    delta2_stations = []

    for key in voc_distances.keys():
        if str(station_id) in key:
            distance = voc_distances[key]

            if key.split(" ")[0] == str(station_id):
                other=key.split(" ")[1]
            elif key.split(" ")[1] == str(station_id):
                other=key.split(" ")[0]
            
            if distance<=1:
                delta0_stations.append(other)         
            elif distance<=2:
                delta1_stations.append(other)
            elif distance<=3:
                delta2_stations.append(other)
        
    distance_dictionary[station_id].append(list(delta0_stations))
    distance_dictionary[station_id].append(list(delta1_stations))
    distance_dictionary[station_id].append(list(delta2_stations))


In [9]:
#Questo dizionario contiene come chiave tutte le stazioni e come valori corrisopondenti tre liste contenenti
# le stazioni che distantano dalla stazione della chiave rispettivamente <1 km, tra 1 e  

# distance_dictionary

In [10]:
# distance_dictionary[63]

### Testo i pattern estratti

In [3]:
### Parametri considerati per la costruzione dei dataset della classificazione
time_interval = 30
window_width = 5

#Parametri usati per l'estazione dei pattern
space_interval = 1000

#Parametri usati per il filtraggio dei pattern
conf_threshold = 50 #valore intero (%)
sup_threshold = 200

#Flag utile a considerare soltanto i pattern contenenti esclusivamente QuasiPiena
filter_normal = False

#Numero di pattern da metchare
num_tot_pat = 1

#Lista di fasce temporali da considerare
time_slots = [
    "0-6", 
    "6-10", 
    "10-14", 
    "14-17", 
    "17-20", 
    "20-24"
]

In [17]:
for slot in time_slots:
    print(f"---- TIME SLOT {slot} ----\n")
    #Se considero soltanto i pattern contenenti soltanto QuasiPiena
    if filter_normal:
        pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/Time_slots/filtered_results_{slot}_({conf_threshold}%-{sup_threshold})_StateChange_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
        output_path = f'Test_results/{time_interval}min/Time_slots/results_{slot}_almostFull_{num_tot_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'
    else:
        pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/Time_slots/filtered_results_{slot}_({conf_threshold}%-{sup_threshold})_StateChange_Normal_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
        output_path = f'Test_results/{time_interval}min/Time_slots/results_{slot}_Normal_almostFull_{num_tot_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'

    file = open(output_path, "w")
    file.write(f'PATTERN TESING RESULTS:\n\n')
    tot_fp = 0
    tot_tp = 0
    tot_fn = 0
    tot_tn = 0


    for station_id in SFancisco_stations:

        test_path = Path(f'../Traditional_classification/datasets/{time_interval}_{window_width}/Train_test_with_time/station{station_id}_test.csv')
        test_df = pd.read_csv(test_path)
        
        time_slot_test_df = test_df.set_index("time", drop=True)
        time_slot_test_df.index=pd.to_datetime(time_slot_test_df.index)
        start_time= slot.split('-')[0]
        end_time = slot.split('-')[1]
        if end_time != '24':       
            time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', f'{end_time}:00')
        else:
            time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', '23:59')

        y_test = time_slot_test_df["status"]
        y_pred = []

        #itero le righe del test set
        for index, row in time_slot_test_df.iterrows():
            num_row=0
            num_pat_matched = 0
            #print(row['docks_av_41_T4'], row['status'])
            prediction = "N"

            #apro il file ed itero su tutti i pattern
            with open(pattern_path, "r") as ifile:
                for line in ifile:
                    num_row+=1
                    #elimino gli header del file
                    if "[[[" not in line:
                        continue

                    flag = '' # sarà usaro per la condizione di uscita se una regola non metcha

                    #Trasformo la linea testuale in una lista
                    line_list = ast.literal_eval(line)

#                     #Prelevo il conseguente
#                     last = line_list[0][0][-1]

                    #elimino il conseguente
                    prior = line_list[0][0][:-1]

                    #Prelevo l'ultimo elemento
                    last = prior[-1]

                    #Ricavo l'istante temporale dell'ultimo slot che corrisponderà all'istante 0 nella tabella del test set
                    curr_time_slot = int(last[0].split('_')[1][1])

                    #Itero gli slot di tempo presenti nell'antecedente
                    for item in prior:
        #                 print(f"1_{flag}")

                        matched_neigh = 0

                        #controllo se è già stata settata la condizione di uscita
                        if flag == "next_line":
                            break

                        #ricavo la stringa contenuta nell'i-esimo slot e la splitto per ottenere la lista dei controlli da fare
                        check_list = item[0].split(",")

                        #itero sulla lista contenuta nello stesso slot temporale
                        for el in check_list:
        #                     print(f"2_{flag}")
                            #controllo se è già stato settata la condizione di uscita
                            if flag == "next_line":
                                break

                            el_status = el.split('_')[0]
                            el_time_slot = int(el.split('_')[1][1])
                            el_space_slot = int(el.split('_')[2])     

                            feature_time_slot = curr_time_slot - el_time_slot

                            #se la stazione è quella di riferimento
                            if el_space_slot == 0:

                                #A partire da queste info costruisco la feature da andare a controllare
                                feature_str = f"docks_av_{station_id}_T{feature_time_slot}"

        #                         print(feature_str)

                                #Se la regola non è verificata, passo alla prossima
                                if (el_status == "QuasiPiena" and row[feature_str] > 2) or (el_status == "Normal" and row[feature_str] <= 2):
                                    flag = "next_line"
        #                             print(f"3_{flag}")
                                    break


                            #se invece si tratta di una delle stazioni vicine:
                            else:
                                neighbors = distance_dictionary[station_id][el_space_slot-1]
                                for neigh_id in neighbors:
                                    #inizialmente lo stesso al caso negativo
                                    flag = "next_line"

                                    #se il vicino corrente ha già verificato la regola, passo al vicino successivo 
                                    # (utile nei casi in cui si hanno piu regole nello stesso delta spaziale)
                                    if neigh_id == matched_neigh:
                                        continue

                                    feature_str = f"docks_av_{neigh_id}_T{feature_time_slot}"
        #                             print(feature_str)

                                    #se il vicino corrente non rispetta il pattern, passo al vicino successivo
                                    if (el_status == "QuasiPiena" and row[feature_str] > 2) or (el_status == "Normal" and row[feature_str] <= 2):
                                        continue

                                    #se invece viene trovato un vicino che rispetta il pattern, rimetto il flag ='' ed esco dal ciclo  
                                    flag=''
                                    matched_neigh= neigh_id
                                    break

                                if flag == "next_line":
                                    break


                    #se a questo punto il flag non è stato settato a "next_line", vuol dire che nessuna delle condizioni sfavorevoli 
                    # precedenti è stata verificata, quindi il pattern ha metchato. 
                    if flag != "next_line":
                        #incremento il numero di pattern verificati
                        num_pat_matched +=1

                        #se i numero di pattern verificati è uguale al numero di pattern da verificare desiderato
                        if num_pat_matched == num_tot_pat: 
                            # Setto pred a "QP" ed esco
                            prediction = "QP"
            #                 print(f'PATTERN MATCHED NUM: {num_row}')
                            break

            y_pred.append(prediction)


        cm = confusion_matrix(y_test, y_pred, labels=["N", "QP"])

        str_= f'PATTERN TEST FOR STATION {station_id}' + '\n'
        str_ += f'Confusion matrix:' + '\n'
        str_ += str(cm) + '\n'

        tn, fp, fn, tp = cm.ravel()
        str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

        test_accuracy = (tn + tp) / (tn + fp + fn + tp)
        #test_recall = (tp) / (tp + fn)
        #test_precision = (tp) / (tp + fp)
        test_recall = recall_score(y_test, y_pred, pos_label='QP', zero_division=0)
        test_precision = precision_score(y_test, y_pred, pos_label='QP', zero_division=0)
        test_f1_score = f1_score(y_test, y_pred, pos_label='QP', zero_division=0)

        str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; f1_score= {test_f1_score}' +'\n\n'
        str_+= "-"*10 +'\n\n'

        tot_fp += fp
        tot_tp += tp
        tot_fn += fn
        tot_tn += tn

        file.write(str_)
#         print(str_)

    avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
    avg_recall = (tot_tp) / (tot_tp + tot_fn)
    if (tot_tp + tot_fp) != 0.0:
        avg_precision = (tot_tp) / (tot_tp + tot_fp)
    else:
        avg_precision = 0.0
    
    if avg_recall!= 0.0 and avg_precision!= 0.0:
        avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))
    else:
        avg_f1_score = 0.0

    avg_str = "AVERAGE VALUES FOR PATTERN TEST:\n"
    avg_str += "Confusion matrix:\n"
    avg_str += f"[[{tot_tn} {tot_fp}]\n[{tot_fn} {tot_tp}]]\n"
    avg_str += f"tot_tp={tot_tp}, tot_fn={tot_fn}, tot_fp={tot_fp}, tot_tn={tot_tn}\n"
    avg_str += f"accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}\n\n"
    print(avg_str)
    file.write(avg_str)
    file.close()


---- TIME SLOT 0-6 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[73709 236]
[170 3445]]
tot_tp=3445, tot_fn=170, tot_fp=236, tot_tn=73709
accuracy=0.9947653429602888; recall=0.9529737206085753; precision=0.9358869872317305; f1_score=0.9443530701754383


---- TIME SLOT 6-10 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[51702 135]
[1433 245]]
tot_tp=245, tot_fn=1433, tot_fp=135, tot_tn=51702
accuracy=0.970699803793329; recall=0.14600715137067938; precision=0.6447368421052632; f1_score=0.23809523809523808


---- TIME SLOT 10-14 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[51665 511]
[475 1109]]
tot_tp=1109, tot_fn=475, tot_fp=511, tot_tn=51665
accuracy=0.9816592261904762; recall=0.7001262626262627; precision=0.6845679012345679; f1_score=0.6922596754057428


---- TIME SLOT 14-17 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[40200 404]
[380 631]]
tot_tp=631, tot_fn=380, tot_fp=404, tot_tn=40200
accuracy=0.9811606391925988; recall=0.624134

#### Tabella contenente tutti i risultati considerando un intervallo di 30 minuti 

In [18]:
current_filter_normal = True

if current_filter_normal:
    time_slots_dict ={
    "0-6" : [1, 15, 18, 20, 25],     # 22 pattern totali
    "6-10" : [1, 15, 18, 20, 25],    # 20 pattern totali
    "10-14" : [1, 15, 20, 25, 30],     # 41 pattern totali
    "14-17" : [1, 15, 20, 25, 30],     # 32 pattern totali
    "17-20" : [1, 15, 20, 25, 30, 35],     # 45 pattern totali
    "20-24" : [1, 15, 18, 20, 25]    # 21 pattern totali
    }
    best_results ={
        "0-6" : [18],     # 22 pattern totali
        "6-10" : [15],    # 20 pattern totali
        "10-14" : [25],     # 41 pattern totali
        "14-17" : [25],     # 32 pattern totali
        "17-20" : [25],     # 45 pattern totali
        "20-24" : [15]    # 21 pattern totali
    }
    
else: 
    time_slots_dict ={
        "0-6" : [1, 15, 18, 20, 25],     # 287 pattern totali(40%), 182(50%), 113(55%), 71(60%)
        "6-10" : [1, 15, 18, 20, 25],    # 159 pattern totali(40%), 4(50%), 0(55%), 0(60%)  
        "10-14" : [1, 15, 20, 25, 30],     # 306 pattern totali(40%), 57(50%), 11(55%), 3(60%) 
        "14-17" : [1, 15, 20, 25, 30],     # 287 pattern totali(40%), 5(50%), 0(55%), 0(60%) 
        "17-20" : [1, 15, 20, 25, 30],     # 323 pattern totali(40%), 133(50%), 21(55%), 0(60%) 
        "20-24" : [1, 15, 20, 25, 30]    # 301 pattern totali(40%), 177(50%), 88(55%), 53(60%) 
    } 
    best_results ={
        "0-6" : [1],     
        "6-10" : [1],    
        "10-14" : [30],     
        "14-17" : [1],     
        "17-20" : [30],     
        "20-24" : [20]    
    }
    
# selezionare il numero di patern, la soglia di confidenza e la soglia di supporto desiderati per stampare tutti 
# i risultati ottenuti per le diverse fasce
current_conf_threshold = 40
current_sup_threshold = 0
num_pattern_list = [1] 

rows = []
result_matrix = []
columns = ["avg_recall", "avg_precision", "avg_f1_score"]

for slot in time_slots_dict.keys():
#     for num_pat in num_pattern_list:
    for num_pat in best_results[slot]:
#     for num_pat in time_slots_dict[slot]:
        if current_filter_normal:
            status_str= "QP"
            input_file_name = f"results_{slot}_almostFull_{num_pat}match_{time_interval}min_{window_width}_({current_conf_threshold}%-{current_sup_threshold}).txt"
        else:
            status_str= "N/QP"
            input_file_name = f"results_{slot}_Normal_almostFull_{num_pat}match_{time_interval}min_{window_width}_({current_conf_threshold}%-{current_sup_threshold}).txt"
        
        input_file_path = f"Test_results/{time_interval}min/Time_slots/{input_file_name}"

        row = f"{status_str}_conf{current_conf_threshold}%_{slot}_{num_pat}match"
        rows.append(row)
        with open(input_file_path, "r") as file:
            lines = file.read().splitlines()
            last_line = lines[-2]
#             print (last_line)
            results = last_line.split(';')
            result_values = []
            for result in results:
                
                value = round(float(result.split('=')[1]), 4)
                result_values.append(value)
            result_matrix.append(result_values[1:])

results_table = pd.DataFrame(result_matrix, columns=columns, index=pd.Index(rows))
print(results_table.to_latex(bold_rows=True))
results_table

\begin{tabular}{lrrr}
\toprule
{} &  avg\_recall &  avg\_precision &  avg\_f1\_score \\
\midrule
\textbf{QP\_conf40\%\_0-6\_18match  } &      0.0122 &         0.9362 &        0.0240 \\
\textbf{QP\_conf40\%\_6-10\_15match } &      0.0083 &         0.6087 &        0.0165 \\
\textbf{QP\_conf40\%\_10-14\_25match} &      0.0126 &         0.9091 &        0.0249 \\
\textbf{QP\_conf40\%\_14-17\_25match} &      0.0049 &         1.0000 &        0.0098 \\
\textbf{QP\_conf40\%\_17-20\_25match} &      0.0066 &         1.0000 &        0.0132 \\
\textbf{QP\_conf40\%\_20-24\_15match} &      0.0168 &         0.8857 &        0.0331 \\
\bottomrule
\end{tabular}



,avg_recall,avg_precision,avg_f1_score
QP_conf40%_0-6_18match,0.0122,0.9362,0.0240
QP_conf40%_6-10_15match,0.0083,0.6087,0.0165
QP_conf40%_10-14_25match,0.0126,0.9091,0.0249
QP_conf40%_14-17_25match,0.0049,1.0000,0.0098
QP_conf40%_17-20_25match,0.0066,1.0000,0.0132
QP_conf40%_20-24_15match,0.0168,0.8857,0.0331


### Valuto le prestazioni generali del modello, considerando alcuni valori di default per il numero minimo di pattern. In questo modo possiamo vedere come si comporta generalmente il modello.

Vado quindi a ricavare e sommare le varie matrici di confusione iterando sulle diverse fasce orarie, per i diversi valori di default per il numero minimo di pattern.

In [5]:
current_filter_normal = False
current_conf_threshold = 50
current_sup_threshold = 0
current_time_slots = [
    "0-6", 
    "6-10", 
    "10-14", 
    "14-17", 
    "17-20", 
    "20-24"
]

total_conf_matrix = {}
result_matrix = []
default_num_pat =[1, 15, 18, 20, 25, 30] 
rows = []

for num_pat in default_num_pat: 
    print(f"NUMERO MINIMO DI PATTERN:{num_pat}")
    if current_filter_normal:
        rows.append(f"QP_conf{current_conf_threshold}_{num_pat}match")
        status= "almostFull"
    else:
        rows.append(f"N/QP_conf{current_conf_threshold}_{num_pat}match")
        status = "Normal_almostFull"
    all_fp=0
    all_tp=0
    all_fn=0
    all_tn=0
        
    for slot in current_time_slots:
        input_file_name = f"results_{slot}_{status}_{num_pat}match_{time_interval}min_{window_width}_({current_conf_threshold}%-{current_sup_threshold}).txt"
        input_file_path = f"./Test_results/{time_interval}min/Time_slots/{input_file_name}" 
        
        with open(input_file_path, "r") as file:
            lines = file.read().splitlines()
            last_line = lines[-3]
#             print (last_line)
            results = last_line.split(', ')
            result_values = []
            for result in results:
#                 print (result)
                value = int(result.split('=')[1])
                result_values.append(value)
        all_tp+=result_values[0]   
        all_fn+=result_values[1]
        all_fp+=result_values[2]
        all_tn+=result_values[3]
        
    avg_accuracy = round((all_tn + all_tp) / (all_tn + all_fp + all_fn + all_tp), 5)
    avg_recall = round((all_tp) / (all_tp + all_fn), 5)
    if (all_tp + all_fp) != 0.0:
        avg_precision = round((all_tp) / (all_tp + all_fp), 5)
    else:
        avg_precision = 0.0

    if avg_recall!= 0.0 and avg_precision!= 0.0:
        avg_f1_score = round(2*(1/((1/avg_recall)+(1/avg_precision))), 5)
    else:
        avg_f1_score = 0.0

#     result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1_score])
    result_matrix.append([avg_recall, avg_precision, avg_f1_score])
    final_str = f"[[{all_tn} {all_fp}]\n[{all_fn} {all_tp}]]\n"
    final_str += f"all_tp={all_tp}, all_fn={all_fn}, all_fp={all_fp}, all_tn={all_tn}\n"
    final_str += f"accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}\n\n"
    print (final_str)
    total_conf_matrix[num_pat] = [all_tp, all_fn, all_fp, all_tn]

# result_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_f1_score'], index=pd.Index(rows))
result_table = pd.DataFrame(result_matrix, columns=['oa_recall', 'oa_precision', 'oa_f1_score'], index=pd.Index(rows))
# result_table.to_csv(f"./Test_results/{time_interval}min/Time_slots/Overall_results_AllSlots_{status}_conf{current_conf_threshold}_sup{current_sup_threshold}.csv")
print(result_table.to_latex(bold_rows=True))
result_table

NUMERO MINIMO DI PATTERN:1
[[302558 2339]
[3268 7815]]
all_tp=7815, all_fn=3268, all_fp=2339, all_tn=302558
accuracy=0.98226; recall=0.70513; precision=0.76965; f1_score=0.73598


NUMERO MINIMO DI PATTERN:15
[[304093 804]
[7653 3430]]
all_tp=3430, all_fn=7653, all_fp=804, all_tn=304093
accuracy=0.97324; recall=0.30948; precision=0.81011; f1_score=0.44787


NUMERO MINIMO DI PATTERN:18
[[304239 658]
[8254 2829]]
all_tp=2829, all_fn=8254, all_fp=658, all_tn=304239
accuracy=0.9718; recall=0.25526; precision=0.8113; f1_score=0.38834


NUMERO MINIMO DI PATTERN:20
[[304338 559]
[8980 2103]]
all_tp=2103, all_fn=8980, all_fp=559, all_tn=304338
accuracy=0.96981; recall=0.18975; precision=0.79001; f1_score=0.306


NUMERO MINIMO DI PATTERN:25
[[304433 464]
[9641 1442]]
all_tp=1442, all_fn=9641, all_fp=464, all_tn=304433
accuracy=0.96802; recall=0.13011; precision=0.75656; f1_score=0.22204


NUMERO MINIMO DI PATTERN:30
[[304551 346]
[9928 1155]]
all_tp=1155, all_fn=9928, all_fp=346, all_tn=304551
a

,oa_recall,oa_precision,oa_f1_score
N/QP_conf50_1match,0.70513,0.76965,0.73598
N/QP_conf50_15match,0.30948,0.81011,0.44787
N/QP_conf50_18match,0.25526,0.81130,0.38834
N/QP_conf50_20match,0.18975,0.79001,0.30600
N/QP_conf50_25match,0.13011,0.75656,0.22204
N/QP_conf50_30match,0.10421,0.76949,0.18356
